In [261]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import DebertaV2ForSequenceClassification, DebertaV2Tokenizer
import os
from tqdm import tqdm
import re
import os
import matplotlib.pyplot as plt
import time
import datetime
import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForMaskedLM
os.environ["WANDB_DISABLED"] = "true"
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
tqdm.pandas()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [244]:
df_train = pd.read_csv("input/feedback-prize-effectiveness/train.csv")
df_train['text'] = df_train['essay_id'].apply(lambda x: open(f'input/feedback-prize-effectiveness/train/{x}.txt').read())

df_test = pd.read_csv('input/feedback-prize-effectiveness/test.csv')
df_test['text'] = df_test['essay_id'].apply(lambda x: open(f'input/feedback-prize-effectiveness/test/{x}.txt').read())

df_train.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,text
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,"Hi, i'm Isaac, i'm going to be writing about h..."
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,"Hi, i'm Isaac, i'm going to be writing about h..."
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,"Hi, i'm Isaac, i'm going to be writing about h..."
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,"Hi, i'm Isaac, i'm going to be writing about h..."
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,"Hi, i'm Isaac, i'm going to be writing about h..."


In [245]:
def txt2num(x):
    tmp=x["discourse_effectiveness"]
    if tmp=="Ineffective":
        return 0
    elif tmp=="Adequate":
        return 1
    else:
        return 2
df_train["label"]=df_train.apply(txt2num,1)

In [246]:
tokenizer = DebertaV2Tokenizer.from_pretrained('microsoft/deberta-v3-base')
tokenizer.add_special_tokens({"sep_token":"<sep>"})
model = DebertaV2ForSequenceClassification.from_pretrained('microsoft/deberta-v3-base', num_labels=3)

loading file spm.model from cache at /Users/gojiteji/.cache/huggingface/hub/models--microsoft--deberta-v3-base/snapshots/559062ad13d311b87b2c455e67dcd5f1c8f65111/spm.model
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /Users/gojiteji/.cache/huggingface/hub/models--microsoft--deberta-v3-base/snapshots/559062ad13d311b87b2c455e67dcd5f1c8f65111/tokenizer_config.json
loading configuration file config.json from cache at /Users/gojiteji/.cache/huggingface/hub/models--microsoft--deberta-v3-base/snapshots/559062ad13d311b87b2c455e67dcd5f1c8f65111/config.json
Model config DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-base",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positio

In [247]:
df_train["data"]=df_train['discourse_type']+"<sep>"+df_train['discourse_text']+"<sep>"+df_train['text']
df_test["data"]=df_test['discourse_type']+"<sep>"+df_test['discourse_text']+"<sep>"+df_test['text']

In [266]:
def preprocess_tokenize(x):
    return tokenizer(x["data"],padding="max_length", max_length=528)["input_ids"]
df_train["input_ids"]=df_train.parallel_apply(preprocess_tokenize,axis=1)
df_test["input_ids"]=df_test.parallel_apply(preprocess_tokenize,axis=1)

In [267]:
train_data=(Dataset.from_pandas(df_train)).train_test_split(train_size=0.9)
test=Dataset.from_pandas(df_test)

In [231]:
training_args = TrainingArguments(
    output_dir='./models/',
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    label_names=["label"],
    #auto_find_batch_size=True,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    #report_to='wandb',
    learning_rate=5e-3,
    #resume_from_checkpoint=""
)

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=0)
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
      'f1': f1,
      'precision': precision,
      'recall': recall
}
trainer = Trainer(
  model=model,
  tokenizer=tokenizer,
  args=training_args,
  train_dataset=train_data["train"],
  eval_dataset=train_data["test"],
  compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [232]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: text, discourse_id, essay_id, discourse_type, discourse_text, discourse_effectiveness, data. If text, discourse_id, essay_id, discourse_type, discourse_text, discourse_effectiveness, data are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
/Users/gojiteji/opt/miniconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization

  0%|          | 0/3 [00:00<?, ?it/s]

/Users/gojiteji/opt/miniconda3/lib/python3.9/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:720: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_scores = torch.bmm(query_layer, key_layer.transpose(-1, -2)) / torch.tensor(
/Users/gojiteji/opt/miniconda3/lib/python3.9/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:804: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  score += c2p_att / torch.tensor(scale, dtype=c2p_att.dtype)
/Users/gojiteji/opt/miniconda3/lib/python3.9/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:827: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or

{'loss': 3.4389, 'learning_rate': 0.0, 'epoch': 1.0}


  0%|          | 0/3 [00:00<?, ?it/s]

Saving model checkpoint to ./models/checkpoint-3
Configuration saved in ./models/checkpoint-3/config.json


{'eval_runtime': 1.7061, 'eval_samples_per_second': 2.931, 'eval_steps_per_second': 1.758, 'epoch': 1.0}


Model weights saved in ./models/checkpoint-3/pytorch_model.bin
tokenizer config file saved in ./models/checkpoint-3/tokenizer_config.json
Special tokens file saved in ./models/checkpoint-3/special_tokens_map.json
added tokens file saved in ./models/checkpoint-3/added_tokens.json


Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 21.4298, 'train_samples_per_second': 0.233, 'train_steps_per_second': 0.14, 'train_loss': 3.43887996673584, 'epoch': 1.0}


TrainOutput(global_step=3, training_loss=3.43887996673584, metrics={'train_runtime': 21.4298, 'train_samples_per_second': 0.233, 'train_steps_per_second': 0.14, 'train_loss': 3.43887996673584, 'epoch': 1.0})

In [278]:
outputs=model(torch.Tensor(test["input_ids"]).long())
submission = pd.read_csv("input/feedback-prize-effectiveness/sample_submission.csv")
submission[["Ineffective","Adequate","Effective"]]=outputs["logits"].detach().numpy()

In [285]:
submission.to_csv("submission.csv")

,discourse_id,Ineffective,Adequate,Effective
0,a261b6e14276,0.150054,0.203791,-0.096013
1,5a88900e7dc1,0.157853,0.193040,-0.107808
2,9790d835736b,0.157949,0.191996,-0.107358
